### Rules:
- 18 weeks total
- Each team has one bye week from week 4-14
- Each team plays 17 games
    - 6 divisional games: play each divisional rival 2 times, 
    - 1 game is against a team from the other conference
    - the other 10 are random teams in the same conference

In [1]:
import copy, json, random

In [2]:
class NFLTeam():
    def __init__(self, name, conference, division):
        self.name = name
        self.conference = conference
        self.division = division
        self.bye_week = None
        self.games_played = 0
        self.division_games_played = 0
        self.conference_games_played = 0
        self.schedule = {i: None for i in range(1,19)}
        self.schedule_outline = {i: None for i in range(1,19)}
        self.opponents_not_faced = []
        self.required_opponents = {"division": None, "conference": None}

    def __repr__(self):
        return f"{self.name} {self.conference[0]}.{self.division[0].lower()}"

    def __str__(self):
        # return f"{self.name} ({self.conference} {self.division}) - Bye Week: {self.bye_week} - Games Played: {self.games_played}"
        return self.__repr__()
    
    def set_bye_week(self, week_num):
        """assigns week_num (int) as the bye week"""
        self.bye_week = week_num
        self.schedule[self.bye_week] = " -- BYE WEEK --"

    def play_game(self, opponent, week_number, home_indicator):
        self.games_played += 1

        if opponent.conference == self.conference:
            self.conference_games_played += 1
            if opponent.division == self.division:
                self.division_games_played += 1

        # delete the opponent from the list of other teams that this team has not played against
        self.opponents_not_faced.remove(opponent)

        # add this game to the team objects schedule
        self.schedule.append({
            'week': week_number,
            'opponent': opponent,
            'home_game': home_indicator  # You can determine if it's a home game based on your scheduling logic
        })

    def add_game(self, week, opponent):
        self.schedule[int(week)] = opponent

In [168]:
class NFLSchedule():
    """
    Holds the master schedule for the NFL Season, optional parameter weeks set to default = 16
    """
    def __init__(self, all_NFL_teams, weeks = 18):
        self.schedule = {i: None for i in range(1, weeks+1)}
        self.allteams = all_NFL_teams
        self.AFC = [ team for team in self.allteams if "AFC" in team.conference ]
        self.NFC = [ team for team in self.allteams if "NFC" in team.conference ]
        self.teams_played_other_conference = []

    def __str__(self):
        # return json.dumps(self.schedule, indent=4)
        return self.allteams

    def add_matchup(self, week, hometeam: NFLTeam, awayteam: NFLTeam):
        # self.schedule[week] = {"home": hometeam, "away": awayteam}
        # hometeam.add_game
        pass

    def weekly_bye_count(self, eligible_weeks):

        # keep randomly creating the number of teeams on bye for each week until there are 32 bye slots
        weekly_bye_count_list = [random.choice([2,4]) for week in eligible_weeks]
        while sum(weekly_bye_count_list) != 32:
            weekly_bye_count_list =  [random.choice([2,4]) for week in eligible_weeks]

        # add the weeks at start and end of season to fill out the full schedule
        weekly_bye_count_list = [0,0,0] + weekly_bye_count_list + [0,0,0,0]

        # list of how many teams from each conference are on a bye for in each week (index)
        weekly_bye_slots_per_conference = [value//2 for value in weekly_bye_count_list]
        return weekly_bye_slots_per_conference

    def assign_bye_weeks(self):
        """Creates the bye weeks and assign each team's .bye_week attribute
            Bye Week Rules:
                - byes occur from week 4 through weeek 14
                - either 2 or 4 teams are on a bye in any given week
                - must choose same number of teams from the AFC and NFC each week
                - each team has 1 bye week per season

        Args:
            league (list): a list of all NFLTeam objects in the league

        Returns:
            *** NOTE: the returned dict is just a visual aid ***
            (dict): a dictionary with key = week_num (int) and value = list of [team.name, team.conference] for each team on bye that week 
        """
        
        # Byes can only occur from week 4 to week 14 (i.e. range index 3 - 13)
        eligible_weeks = list(range(3, 14)) 
        weekly_bye_slots_per_conference = self.weekly_bye_count(eligible_weeks)

        # Set up a while loop so that if it tries to assign a bye week on a divisional week, it will start over and try again.
        keep_searching = True
        count = 0
        cutoff = 10000
        while keep_searching == True:
            count += 1
            # print(count)
            if count > cutoff:
                break

            # create temp copies of the self attributes that we need for this round
            AFC_teams = self.AFC.copy()
            NFC_teams = self.NFC.copy()
            random.shuffle(AFC_teams)
            random.shuffle(NFC_teams)
            bye_list = {team: None for team in self.allteams}

            # for each week when byes can occur, choose "num_slots" teams who are not in divisional play this week for a bye
            for week_num in eligible_weeks:
                for num_slots in range(weekly_bye_slots_per_conference[week_num]):
                    eligible_teams_afc = [team for team in AFC_teams if team.schedule_outline[week_num] == None]       
                    eligible_teams_nfc = [team for team in NFC_teams if team.schedule_outline[week_num] == None]
                    # print(f"{eligible_teams_afc = }")
                    # print(f"{eligible_teams_nfc = }")

                    if eligible_teams_afc and eligible_teams_nfc:
                        """choose AFC team(s)"""
                        # selected_team_a = random.choice(eligible_teams_afc)
                        selected_team_a = eligible_teams_afc[0]

                        # print(f"{selected_team_a = }")
                        # remove that team form the temp list so we don't pick it again
                        AFC_teams.remove(selected_team_a)
                        # replace the default None value with the weeke_number of their bye
                        bye_list[selected_team_a] = week_num

                        """choose NFC team(s)"""
                        # selected_team_n = random.choice(eligible_teams_nfc)
                        selected_team_n = eligible_teams_nfc[0]

                        # print(f"{selected_team_n = }")
                        # remove that team form the temp list so we don't pick it again
                        NFC_teams.remove(selected_team_n)
                        bye_list[selected_team_n] = week_num
                    else:
                        # if there are not any eligible teams from one of the conferences, then break and try again
                        keep_searching = True
                        break
            
            # print(f"{bye_list.values() = }")

            if all(value is not None for value in bye_list.values()):
                # this means at least one of the teams was not assigned a bye week, so need to try again
                # print("\nSUCCESS!!")
                print(f"{count = }")
                # print(f"{bye_list = }")
                keep_searching = False
                # return count
            else:
                keep_searching = True

        # return count
        """ sometimes it just doesnt work. if thats the case, call the function again recursively to reset the kernel """
        # if count > cutoff:
        #     print("TRYING AGAIN")
        #     self.assign_bye_weeks()
        # else:
        #     for team, byeweek in bye_list.items():
        #         team.set_bye_week(byeweek)

        return count, cutoff

        # print the results for quality control purposes
        # print(bye_list)
        # print(f"{count = }")
        # print({week: [[team, team.conference] for team in self.allteams if team.bye_week == week] for week in eligible_weeks})
        # return bye_week_summary



    def set_schedule_outline(self) -> None:
        """ For each division, set aside 6 weeks for divisional games. add those 6 weeks to each teams sheduule outline
            Then add each team's bye week, continue doing until none of the bye weeks overlap with each teams weeks reserved for divisional games 
        """

        # set aside weeks for divisional play
        divisions = ["North", "South", "East", "West"]
        
        for div in divisions:
            divisional_game_weeks = sorted(random.sample(range(len(self.schedule)), k = 6))
            for team in [t for t in self.allteams if t.division == div]:
                for week_num in divisional_game_weeks:
                    team.schedule_outline[week_num] = "d"

        # set up bye weeks:
        x, cutoff = self.assign_bye_weeks()
        if x >= cutoff:
            print("FAIL")
            # x, _ = self.assign_bye_weeks()
        return x

        

In [169]:
def create_teams() -> list:
    """ Returns a list of the 32 NFLTeam objects, one for each team"""

    # AFC East Teams
    patriots = NFLTeam("New England Patriots", "AFC", "East")
    bills = NFLTeam("Buffalo Bills", "AFC", "East")
    dolphins = NFLTeam("Miami Dolphins", "AFC", "East")
    jets = NFLTeam("New York Jets", "AFC", "East")

    # AFC North Teams
    ravens = NFLTeam("Baltimore Ravens", "AFC", "North")
    steelers = NFLTeam("Pittsburgh Steelers", "AFC", "North")
    browns = NFLTeam("Cleveland Browns", "AFC", "North")
    bengals = NFLTeam("Cincinnati Bengals", "AFC", "North")

    # AFC South Teams
    texans = NFLTeam("Houston Texans", "AFC", "South")
    colts = NFLTeam("Indianapolis Colts", "AFC", "South")
    titans = NFLTeam("Tennessee Titans", "AFC", "South")
    jaguars = NFLTeam("Jacksonville Jaguars", "AFC", "South")

    # AFC West Teams
    chiefs = NFLTeam("Kansas City Chiefs", "AFC", "West")
    broncos = NFLTeam("Denver Broncos", "AFC", "West")
    raiders = NFLTeam("Las Vegas Raiders", "AFC", "West")
    chargers = NFLTeam("Los Angeles Chargers", "AFC", "West")

    # NFC East Teams
    cowboys = NFLTeam("Dallas Cowboys", "NFC", "East")
    washington = NFLTeam("Washington Football Team", "NFC", "East")
    eagles = NFLTeam("Philadelphia Eagles", "NFC", "East")
    giants = NFLTeam("New York Giants", "NFC", "East")

    # NFC North Teams
    packers = NFLTeam("Green Bay Packers", "NFC", "North")
    bears = NFLTeam("Chicago Bears", "NFC", "North")
    vikings = NFLTeam("Minnesota Vikings", "NFC", "North")
    lions = NFLTeam("Detroit Lions", "NFC", "North")

    # NFC South Teams
    buccaneers = NFLTeam("Tampa Bay Buccaneers", "NFC", "South")
    saints = NFLTeam("New Orleans Saints", "NFC", "South")
    panthers = NFLTeam("Carolina Panthers", "NFC", "South")
    falcons = NFLTeam("Atlanta Falcons", "NFC", "South")

    # NFC West Teams
    seahawks = NFLTeam("Seattle Seahawks", "NFC", "West")
    rams = NFLTeam("Los Angeles Rams", "NFC", "West")
    cardinals = NFLTeam("Arizona Cardinals", "NFC", "West")
    sf49ers = NFLTeam("San Francisco 49ers", "NFC", "West")

    # Create a list of all 32 NFLTeam objects
    return [patriots, bills, dolphins, jets, ravens, steelers, browns, bengals,
            texans, colts, titans, jaguars,chiefs, broncos, raiders, chargers,
            cowboys, washington, eagles, giants,packers, bears, vikings, lions,
            buccaneers, saints, panthers, falcons, seahawks, rams, cardinals, sf49ers]

In [170]:
def add_game_to_team(home_team: NFLTeam, away_team: NFLTeam) -> None:
    """Accesses the pre-defined object for both teams and adds this matchup to the team objects scheduule
    Args:
        home_team (class <NFLTeam>): the home team 
        away_team (class <NFLTeam>): the away team 
    """
    # check if its a divisional or conference game
    if home_team.conference == home_team.conference:
        home_team.conference_games_played += 1
        away_team.conference_games_played += 1
        # if in conference, check if also in division.
        if home_team.division == home_team.division:
            home_team.division_games_played += 1
            away_team.division_games_played += 1
    # home_team.s

In [171]:
myleague = NFLSchedule(create_teams(), weeks=18)

x = myleague.set_schedule_outline()
x

count = 1


1

In [181]:
# create 100 leagues, see how many times the scheduuler fails
attempts = 1000
results = [NFLSchedule(create_teams(), weeks=18).set_schedule_outline() for _ in range(attempts)] 

failure_rate = (sum([1 for i in results if i > 10000]) / attempts) * 100.00

failure_rate

count = 3
count = 1
count = 60
count = 27
FAIL
count = 3
count = 1
count = 62
count = 18
count = 736
count = 4
count = 2
count = 3
FAIL
count = 10
FAIL
FAIL
count = 1
count = 3
count = 10
count = 2
count = 2
count = 7
count = 88
count = 89
count = 1
count = 26
count = 6
count = 1
FAIL
count = 2
count = 1
count = 10
FAIL
count = 2
count = 5
count = 10
count = 7
count = 66
count = 1
FAIL
count = 20
count = 1
count = 15
count = 1
FAIL
count = 2
count = 34
count = 1
FAIL
FAIL
count = 29
count = 3
count = 27
FAIL
count = 1
count = 6
count = 2
count = 1
count = 6
FAIL
count = 16
count = 3
count = 7
FAIL
count = 22
count = 1
count = 45
FAIL
count = 8
count = 228
count = 41
count = 118
count = 93
count = 15
FAIL
count = 1
count = 14
FAIL
FAIL
count = 4
count = 14
count = 1
count = 2
count = 1
count = 190
count = 2
count = 1
count = 35
count = 3
count = 27
FAIL
count = 2
count = 1
count = 1
count = 5
count = 311
count = 11
count = 10
FAIL
count = 81
count = 1
count = 29
count = 1
count = 1
coun

12.7

In [84]:
for team in myleague.allteams:
    print(team.schedule_outline, "- - -",  team)

{1: 'd', 2: 'd', 3: None, 4: 'd', 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: 'd', 13: None, 14: 'd', 15: None, 16: 'd', 17: None, 18: None} - - - New England Patriots A.e
{1: 'd', 2: 'd', 3: None, 4: 'd', 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: 'd', 13: None, 14: 'd', 15: None, 16: 'd', 17: None, 18: None} - - - Buffalo Bills A.e
{1: 'd', 2: 'd', 3: None, 4: 'd', 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: 'd', 13: None, 14: 'd', 15: None, 16: 'd', 17: None, 18: None} - - - Miami Dolphins A.e
{1: 'd', 2: 'd', 3: None, 4: 'd', 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: 'd', 13: None, 14: 'd', 15: None, 16: 'd', 17: None, 18: None} - - - New York Jets A.e
{1: None, 2: 'd', 3: 'd', 4: None, 5: None, 6: None, 7: 'd', 8: 'd', 9: None, 10: None, 11: 'd', 12: None, 13: None, 14: None, 15: 'd', 16: None, 17: None, 18: None} - - - Baltimore Ravens A.n
{1: None, 2: 'd', 3: 'd', 4: None, 5: N

In [ ]:
# ASSIGNING BYE WEEKS
myleague.assign_bye_weeks()


In [ ]:
patriots = myleague.allteams[0]
patriots.schedule_outline

In [ ]:

# print("Assigning bye weeks: \n")
# assign_bye_weeks(nfl_teams)

In [ ]:
nfl_teams

In [ ]:
print(len(bengals.opponents_not_faced))

for team in nfl_teams:
    team.opponents_not_faced = [team for team in nfl_teams]
    # team.opponents_not_faced.remove(team.name)

print(len(bengals.opponents_not_faced))


In [ ]:
# bengals.add_game(2,chargers)
bengals.schedule
# patriots.schedule

In [ ]:
myschedule = NFLSchedule(weeks=18)
full_schedule = {}


for week_num in range(0,19):
# for week_num in range(4,5):
    weekly_matchups = []
    weekly_matchups_names = []

    # list of teams in afc and nfc that are playing a game this week
    afc_eligible_teams = [team for team in AFC if team.bye_week != week_num]
    nfc_eligible_teams = [team for team in NFC if team.bye_week != week_num]

    # shuffle the teams to get randomized matchups
    random.shuffle(afc_eligible_teams)
    random.shuffle(nfc_eligible_teams)

    print(f"\n{week_num = }")
    # print(f"{afc_eligible_teams = }")
    # print(f"{nfc_eligible_teams = }")

    for team in nfl_teams:
        opponent = random.choice(team.opponents_not_faced)
        # team.opponents_not_faced.remove(opponent)
        

    # as if every team plays someone from the other conference
    for afc_team, nfc_team in zip(afc_eligible_teams, nfc_eligible_teams):
        weekly_matchups.append({"home": afc_team, "away": nfc_team})
        weekly_matchups_names.append({"home": afc_team.name, "away": nfc_team.name})
    # print(f"{weekly_matchups_names = }")


    # assign that game to each team's 
    for game in weekly_matchups:
        # print(game)
        home_team = game["home"]
        away_team = game["away"]

        print(away_team, "@", home_team)

        home_team.add_game(week_num, away_team)
        away_team.add_game(week_num, home_team)

        # game["home"].add_game(week_num, game["away"])
        # game["away"].add_game(week_num, game["home"])
        
        # add_game_to_team(week_num, game["home"])
        # add_game_to_team(week_num, game["away"])
        
    full_schedule[f"week_{week_num+1}"] = weekly_matchups
    
# full_schedule_json = json.dumps(full_schedule, indent=4)  # indent for pretty printing
# print(full_schedule_json)
# full_schedule


In [ ]:
# [game.name if type(game) == NFLTeam else game for game in bengals.schedule.values()]
# [game.name if type(game) == NFLTeam else game for game in sf49ers.schedule.values()]

[game.name if type(game) == NFLTeam else game for game in patriots.schedule.values()]
# [game.name if type(game) == NFLTeam else game for game in cowboys.schedule.values()]